In [1]:
import huxt as H
import huxt_analysis as HA
import huxt_inputs as Hin
import SIR_HUXt_plots as sirplt
#import sir_huxt_lon as shl
#import sir_huxt_v as shv
#import sir_huxt_width as shw
import sir_huxt_v_width_lon as shvwl

import astropy.units as u
import glob
import os
import h5py
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy.stats as st
import sunpy.coordinates.sun as sn

from sklearn.neighbors import KernelDensity
from astropy.time import Time

In [2]:
# Test the SIR scheme
np.random.seed(19630802)

model = shvwl.setup_uniform_huxt(dt_scale=14)

# Generate a "truth" CME
base_cme = shvwl.get_base_cme()

# Get HUXt solution of this truth CME, and observations from L5
model.solve([base_cme])
cme_truth = model.cmes[0]

observer_lon = -60*u.deg
L5Obs = shvwl.Observer(model, cme_truth, observer_lon, el_min=4.0, el_max=30.0)

fig, ax = sirplt.plot_huxt_with_observer(model.time_out[8], model, L5Obs, add_flank=True, add_fov=True)
fig.savefig('test_truth.png')
plt.close('all')

#############################################
# SHVW - Velocity and width
# Make directory to store this experiment in
dirs = shvwl.get_project_dirs()
output_dir = 'shvw_test'
output_dir = os.path.join(dirs['sir_analysis'], output_dir)
if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

# Run the SIR scheme on this event many times to see how the performance is
n_ens = 5

# Make a guess at the CME initial values 
cme_guess = shvwl.perturb_cme(base_cme)

# Low observational error
observed_cme_flank = L5Obs.compute_synthetic_obs(el_spread=0.1, cadence=1, el_min=4.0, el_max=30.0)

observations = {'observer_lon':observer_lon, 'observed_cme_flank':observed_cme_flank, 'truth_cme_params':cme_truth.parameter_array()}

tag = "shvwl_run_{:03d}".format(0)
shvwl.SIR(model, cme_guess, observations, n_ens, output_dir, tag)

